In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('pre').getOrCreate()

22/07/06 22:26:52 WARN Utils: Your hostname, indhira resolves to a loopback address: 127.0.1.1; using 192.168.100.7 instead (on interface wlp2s0)
22/07/06 22:26:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/06 22:26:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [158]:
df = spark.read.option('header', 'true').option('inferSchema', 'true').csv('../n/OutEnv-hour.csv')

In [159]:
df.count()

8761

In [160]:
df = df.na.drop(subset=["Timestamp","OutEnv_OutdoorAmbTemp"])
df_rdd = df.rdd

In [167]:
df.count()

8672

In [163]:
from pyspark.sql import Row
def pydf_to_str(row, col):
    rd = row.asDict()
    rd[col] = str(rd[col])[0:19]
    nrow = Row(**rd)
    return nrow
def pydf_to_int(row, col):
    rd = row.asDict()
    rd[col] = round(rd[col],0)
    nrow = Row(**rd)
    return nrow


In [168]:
df_rdd = df_rdd.map(lambda x:pydf_to_str(x, 'Timestamp'))
df_rdd = df_rdd.filter(lambda x:x['Timestamp'] >= '2013-07-01').filter(lambda x:x['Timestamp'] <'2013-08-01')

In [170]:
df_rdd.take(1)

[Row(Timestamp='2013-07-01 00:00:00', OutEnv_OutdoorAmbTemp=22.756409988485963, OutEnv_RooftopWindDirection=175.28333333333333, OutEnv_RooftopWindSpeed=1.8133333333333332)]

In [171]:
df_rdd = df_rdd.map(lambda x:pydf_to_int(x, 'OutEnv_OutdoorAmbTemp'))

In [172]:
df_rdd.take(1)

[Row(Timestamp='2013-07-01 00:00:00', OutEnv_OutdoorAmbTemp=23.0, OutEnv_RooftopWindDirection=175.28333333333333, OutEnv_RooftopWindSpeed=1.8133333333333332)]

In [173]:
import math
df_rdd2 = df_rdd.map(lambda x: 
    (x[1], int(str(x[0])[8:10]))
)
df_rdd2.collect()

In [217]:
df_rdd3 = df_rdd2.groupByKey()
df_rdd3.collect()

[(23.0, <pyspark.resultiterable.ResultIterable at 0x7f97de711cd0>),
 (22.0, <pyspark.resultiterable.ResultIterable at 0x7f97de667b20>),
 (24.0, <pyspark.resultiterable.ResultIterable at 0x7f97de667f10>),
 (26.0, <pyspark.resultiterable.ResultIterable at 0x7f97de6678e0>),
 (27.0, <pyspark.resultiterable.ResultIterable at 0x7f97de667fd0>),
 (25.0, <pyspark.resultiterable.ResultIterable at 0x7f97df85db80>),
 (28.0, <pyspark.resultiterable.ResultIterable at 0x7f97df85d700>),
 (29.0, <pyspark.resultiterable.ResultIterable at 0x7f97df85da60>),
 (30.0, <pyspark.resultiterable.ResultIterable at 0x7f97df85d340>),
 (31.0, <pyspark.resultiterable.ResultIterable at 0x7f97df85d9a0>),
 (32.0, <pyspark.resultiterable.ResultIterable at 0x7f97df85d2b0>),
 (21.0, <pyspark.resultiterable.ResultIterable at 0x7f97df85dd90>),
 (20.0, <pyspark.resultiterable.ResultIterable at 0x7f97df85dc10>),
 (19.0, <pyspark.resultiterable.ResultIterable at 0x7f97de661850>),
 (33.0, <pyspark.resultiterable.ResultIterable a

In [218]:
df_rdd3 = df_rdd3.map(lambda x : (x[0], set(x[1]), 30*"0"))
df_rdd3.collect()

[(23.0,
  {1,
   2,
   3,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   15,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31},
  '000000000000000000000000000000'),
 (22.0,
  {1, 3, 7, 8, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31},
  '000000000000000000000000000000'),
 (24.0,
  {1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   13,
   14,
   15,
   20,
   21,
   22,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31},
  '000000000000000000000000000000'),
 (26.0,
  {1,
   2,
   3,
   4,
   5,
   6,
   8,
   9,
   10,
   11,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   26,
   27,
   28,
   29,
   30,
   31},
  '000000000000000000000000000000'),
 (27.0,
  {1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   11,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   21,
   22,
   23,
   24,
   26,
   27,
   28,
   29,
   30},
  '000000000000000000000000000000'),
 (25.0,
  {1,
  

In [219]:
def make_bitmap(x):
    s = list(31*"0")
    for i in x:
        s[i-1] = "1"
    return "".join(s)

In [220]:
df_rdd3 = df_rdd3.map(lambda x:(x[0], x[1], make_bitmap(x[1])))
df_rdd3.collect()

[(23.0,
  {1,
   2,
   3,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   15,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31},
  '1111101111111010000011111111111'),
 (22.0,
  {1, 3, 7, 8, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31},
  '1010001100111000000001111111111'),
 (24.0,
  {1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   13,
   14,
   15,
   20,
   21,
   22,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31},
  '1111111111101110000111011111111'),
 (26.0,
  {1,
   2,
   3,
   4,
   5,
   6,
   8,
   9,
   10,
   11,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   26,
   27,
   28,
   29,
   30,
   31},
  '1111110111101111111111110111111'),
 (27.0,
  {1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   11,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   21,
   22,
   23,
   24,
   26,
   27,
   28,
   29,
   30},
  '1111111110101111111011110111110'),
 (25.0,
  {